---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [359]:
import pandas as pd
import numpy as np
import datetime as dt
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [360]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    import pandas as pd
    import numpy as np
    import datetime as dt
    from scipy.stats import ttest_ind
    
    # Use this dictionary to map state names to two letter acronyms
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    
    uni_town = pd.read_csv("university_towns.txt", sep='delimiter', header=None)

    sep = '['
    for i in range(0,len(uni_town)):
        uni_town.iloc[i][0] = uni_town.iloc[i][0].split(sep, 1)[0]
        
    sep = '('
    for i in range(0,len(uni_town)):
        uni_town.iloc[i][0] = uni_town.iloc[i][0].split(sep, 1)[0]
        
    dict_state = {}
    State = []
    RegionName = []

    
    for i in range(0,len(uni_town)):
        if uni_town.iloc[i][0] in states.values():
            current_state = uni_town.iloc[i][0]
        else:
            State.append(current_state)
            RegionName.append(uni_town.iloc[i][0][:-1])
    dict_state = {"State" : State, "RegionName" : RegionName}
    
    df = pd.DataFrame.from_dict(dict_state)
    df = df[["State", "RegionName"]]
    
    return df

In [361]:
check_df = get_list_of_university_towns()
check_df

c:\users\rbosshard\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [362]:
a = check_df.iloc[0][1]
a

'Auburn'

In [363]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    import pandas as pd
    import numpy as np
    import datetime as dt
    from scipy.stats import ttest_ind
    
    gdp = pd.read_excel('gdplev.xls')
    
    gdp.columns = ["DELETEAnnual", "DELETEGDP in billions of current dollars",
               "DELETEGDP in billions of chained 2009 dollars", "delete", "Quarterly", "DELGDP in billions of current dollars",
               "GDP in billions of chained 2009 dollars", "delete2"]
    gdp = gdp.drop(gdp.index[0:219])
    gdp = gdp.reset_index()
    gdp = gdp.drop('index', 1)
    gdp = gdp.drop('delete', 1)
    gdp = gdp.drop('delete2', 1)
    gdp = gdp.drop('DELETEAnnual', 1)
    gdp = gdp.drop('DELETEGDP in billions of current dollars', 1)
    gdp = gdp.drop('DELETEGDP in billions of chained 2009 dollars', 1)
    gdp = gdp.drop('DELGDP in billions of current dollars', 1)
    
    for i in range(0,len(gdp)):
        try:
            if gdp["GDP in billions of chained 2009 dollars"][i] < gdp["GDP in billions of chained 2009 dollars"][i-1]:
                if gdp["GDP in billions of chained 2009 dollars"][i-1] < gdp["GDP in billions of chained 2009 dollars"][i-2]:
                    rec_start = gdp["Quarterly"][i]
                    break
        except KeyError:
            continue
    
    return rec_start

In [364]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    import pandas as pd
    import numpy as np
    from scipy.stats import ttest_ind
    
    gdp = pd.read_excel('gdplev.xls')
    
    gdp.columns = ["DELETEAnnual", "DELETEGDP in billions of current dollars",
               "DELETEGDP in billions of chained 2009 dollars", "delete", "Quarterly", "DELGDP in billions of current dollars",
               "GDP in billions of chained 2009 dollars", "delete2"]
    gdp = gdp.drop(gdp.index[0:219])
    gdp = gdp.reset_index()
    gdp = gdp.drop('index', 1)
    gdp = gdp.drop('delete', 1)
    gdp = gdp.drop('delete2', 1)
    gdp = gdp.drop('DELETEAnnual', 1)
    gdp = gdp.drop('DELETEGDP in billions of current dollars', 1)
    gdp = gdp.drop('DELETEGDP in billions of chained 2009 dollars', 1)
    gdp = gdp.drop('DELGDP in billions of current dollars', 1)
    

    for i in range(0,len(gdp)):
        try:
            if gdp["GDP in billions of chained 2009 dollars"][i] < gdp["GDP in billions of chained 2009 dollars"][i-1]:
                if gdp["GDP in billions of chained 2009 dollars"][i-1] < gdp["GDP in billions of chained 2009 dollars"][i-2]:
                    rec_start = gdp["Quarterly"][i]
                    rec_start_index = i
                    break
        except KeyError:
            continue
    
    for i in range(rec_start_index,len(gdp)):
        try:
            if gdp["GDP in billions of chained 2009 dollars"][i-1] > gdp["GDP in billions of chained 2009 dollars"][i-2]:
                if gdp["GDP in billions of chained 2009 dollars"][i] > gdp["GDP in billions of chained 2009 dollars"][i-1]:
                    rec_end = gdp["Quarterly"][i]
                    break
        except KeyError:
            continue  
    
    return rec_end

In [365]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    import pandas as pd
    import numpy as np
    import datetime as dt
    from scipy.stats import ttest_ind
    
    gdp = pd.read_excel('gdplev.xls')
    
    gdp.columns = ["DELETEAnnual", "DELETEGDP in billions of current dollars",
               "DELETEGDP in billions of chained 2009 dollars", "delete", "Quarterly", "DELGDP in billions of current dollars",
               "GDP in billions of chained 2009 dollars", "delete2"]
    gdp = gdp.drop(gdp.index[0:219])
    gdp = gdp.reset_index()
    gdp = gdp.drop('index', 1)
    gdp = gdp.drop('delete', 1)
    gdp = gdp.drop('delete2', 1)
    gdp = gdp.drop('DELETEAnnual', 1)
    gdp = gdp.drop('DELETEGDP in billions of current dollars', 1)
    gdp = gdp.drop('DELETEGDP in billions of chained 2009 dollars', 1)
    gdp = gdp.drop('DELGDP in billions of current dollars', 1)
    

    for i in range(0,len(gdp)):
        try:
            if gdp["GDP in billions of chained 2009 dollars"][i] < gdp["GDP in billions of chained 2009 dollars"][i-1]:
                if gdp["GDP in billions of chained 2009 dollars"][i-1] < gdp["GDP in billions of chained 2009 dollars"][i-2]:
                    rec_start = gdp["Quarterly"][i]
                    rec_start_index = i
                    break
        except KeyError:
            continue
    
    for i in range(rec_start_index,len(gdp)):
        try:
            if gdp["GDP in billions of chained 2009 dollars"][i-1] > gdp["GDP in billions of chained 2009 dollars"][i-2]:
                if gdp["GDP in billions of chained 2009 dollars"][i] > gdp["GDP in billions of chained 2009 dollars"][i-1]:
                    rec_end = gdp["Quarterly"][i]
                    rec_end_index = i
                    break
        except KeyError:
            continue  

    for i in range(rec_start_index,rec_end_index):
        try:
            if gdp["GDP in billions of chained 2009 dollars"][i] < gdp["GDP in billions of chained 2009 dollars"][i-1]:
                rec_bottom = gdp["Quarterly"][i]
                
        except KeyError:
            continue  
    
    return rec_bottom

In [366]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    import numpy as np
    import datetime as dt
    from scipy.stats import ttest_ind
    
    all_homes = pd.read_csv('City_Zhvi_AllHomes.csv')
    all_homes = all_homes.drop(['RegionID', 'SizeRank', 'Metro', 'CountyName'], axis=1)
    all_reset_index = all_homes.reset_index()
    
    normalized_df = all_reset_index.set_index(['State', 'RegionName', 'index']).stack(dropna=False)
    normalized_df = normalized_df.reset_index()
    normalized_df = normalized_df.rename(columns={"level_3": "date", 0: "home_median"})
    normalized_df.loc[:, 'date'] = pd.to_datetime(normalized_df['date'], format='%Y-%m')
    normalized_df = normalized_df[normalized_df["date"] >= dt.date(2000, 1, 1)]
    
    normalized_df.loc[:, 'quarter'] = normalized_df['date'].dt.quarter
    normalized_df.loc[:, 'year'] = normalized_df['date'].dt.year

    normalized_df.loc[:, 'yq']=normalized_df["year"].map(str) + "q" + normalized_df["quarter"].map(str)
    
    normalized_df = normalized_df.drop(['date', "quarter", "year"], axis=1)
    
    mean_df = normalized_df.groupby(['State', 'RegionName', 'index', 'yq']).mean()
    
    mean_unstacked_df = mean_df.unstack()

    mean_unstacked_df.columns = mean_unstacked_df.columns.droplevel()
    mean_unstacked_df = mean_unstacked_df.reset_index()
    del mean_unstacked_df.columns.name
    
    mean_unstacked_df = mean_unstacked_df.drop('index', axis=1)
    mean_unstacked_df = mean_unstacked_df.set_index(["State", "RegionName"])
    return mean_unstacked_df


In [367]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    import pandas as pd
    import numpy as np
    import datetime as dt
    from scipy.stats import ttest_ind
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 
     'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 
     'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 
     'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 
     'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 
     'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 
     'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 
     'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 
     'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 
     'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

    df =  convert_housing_data_to_quarters()
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()

    
    df1 = df[[rec_start, rec_bottom]]
    
    df1["change"] = df1[rec_start] / df1["2009q2"]
    
    
    df2 = get_list_of_university_towns()
    df3 = pd.DataFrame(states, index=states.keys())
    df3 = df3.stack()
    df3 = df3.reset_index()
    df3.columns = ["delete", "ST", "State"]
    del df3["delete"]
    df4 = pd.merge(df2, df3, on="State")
    df4 = df4.drop_duplicates()
    df4 = df4.reset_index()
    del df4["index"]
    df4 = df4[["ST", "RegionName", "State"]]
    df4["university_town"] = 1
    df4 = df4.set_index(["ST", "RegionName"])
    del df4["State"]
    
    df_a = df1.reset_index()
    df_a.columns = ["ST", "RegionName", "2008q4", "2009q2", "change"]
    df_b = df4.reset_index()

    df5 = pd.merge(df_a, df_b, how="left", on=["ST", "RegionName"])
    df5["university_town"].fillna(value=0, inplace=True)
    
    df5["university_town_value"] = df5["university_town"]*df5["change"]
    df_uni_town = df5[["university_town_value"]]
    df_uni_town = df_uni_town[(df_uni_town.T != 0).any()]
    df_uni_town.dropna(inplace=True)
    df_uni_town = df_uni_town.reset_index()
    del df_uni_town["index"]
    
    df5["non_uni_value"] = np.where(df5['university_town'] == 0, df5["change"], 0)
    
    df_non_uni = df5[["non_uni_value"]]
    df_non_uni = df_non_uni[(df_non_uni.T != 0).any()]
    df_non_uni.dropna(inplace=True)
    df_non_uni = df_non_uni.reset_index()
    del df_non_uni["index"]

    ttest_ind(df_uni_town, df_non_uni)
    
    a = ttest_ind(df_uni_town, df_non_uni)
    p = a[1][0]
    t = a[0][0]
    
    different = ""
    better = ""
    
    if abs(t) > 2:
        if p<0.01:
            different = True
        else:
            different = False
    else:
        different = False
    
    if df_non_uni.mean()[0] < df_uni_town.mean()[0]:
        better = "non-university town"
    else:
        better = "university town"
            
    return (different, p, better)

In [368]:
run_ttest()

c:\users\rbosshard\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\rbosshard\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


(False, 0.02617906891942761, 'university town')

In [369]:
rec_start = get_recession_start()
rec_start

'2008q4'

In [370]:
rec_bottom = get_recession_bottom()
rec_bottom

'2009q2'

In [371]:
df =  convert_housing_data_to_quarters()

In [372]:
df1 = df[["2008q4", "2009q2"]]

In [373]:
df1["change"] = df1["2008q4"] / df1["2009q2"]

c:\users\rbosshard\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [374]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 
 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 
 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 
 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 
 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 
 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 
 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 
 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 
 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 
 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

df2 = get_list_of_university_towns()
df3 = pd.DataFrame(states, index=states.keys())
df3 = df3.stack()
df3 = df3.reset_index()
df3.columns = ["delete", "ST", "State"]
del df3["delete"]
df4 = pd.merge(df2, df3, on="State")
df4 = df4.drop_duplicates()
df4 = df4.reset_index()
del df4["index"]
df4 = df4[["ST", "RegionName", "State"]]
df4["university_town"] = 1
df4 = df4.set_index(["ST", "RegionName"])
del df4["State"]


c:\users\rbosshard\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [375]:
df_a = df1.reset_index()
df_a.columns = ["ST", "RegionName", "2008q4", "2009q2", "change"]
df_b = df4.reset_index()

df5 = pd.merge(df_a, df_b, how="left", on=["ST", "RegionName"])
df5["university_town"].fillna(value=0, inplace=True)


In [376]:
df5["university_town_value"] = df5["university_town"]*df5["change"]
df_uni_town = df5[["university_town_value"]]
df_uni_town = df_uni_town[(df_uni_town.T != 0).any()]
df_uni_town.dropna(inplace=True)
df_uni_town = df_uni_town.reset_index()
del df_uni_town["index"]


In [377]:
df5["non_uni_value"] = np.where(df5['university_town'] == 0, df5["change"], 0)

In [378]:
df_non_uni = df5[["non_uni_value"]]
df_non_uni = df_non_uni[(df_non_uni.T != 0).any()]
df_non_uni.dropna(inplace=True)
df_non_uni = df_non_uni.reset_index()
del df_non_uni["index"]


In [379]:
ttest_ind(df_uni_town, df_non_uni)

Ttest_indResult(statistic=array([-2.22387944]), pvalue=array([0.02617907]))

In [380]:
df_uni_town.mean()

university_town_value    1.021239
dtype: float64

In [381]:
df_non_uni.mean()[0]

1.0295129927204294

In [382]:
a = ttest_ind(df_uni_town, df_non_uni)
p = a[1][0]
p

0.02617906891942761

In [383]:
t = a[0][0]
t

-2.223879441141932